# **Universidade Tiradentes**

Pós-Graduação em Ciência de Dados e Inteligência Artificial- Turma 2024/001

Disciplina: Linguagens Python e R com Foco em Análise de Dados

Aluno(s): Cleiton Alves Fraga, Rodolfo Brandão

**Instruções gerais**  
- Use este notebook como **template** para responder às questões.  
- Utilizem as bibliotecas scipy.stats e statsmodels para auxiliar nas soluções
das questões propostas.
- Comente seu código e **justifique** suas respostas numéricas com frases curtas.  
- O banco de dados é o **microdados ENEM (Nordeste)** previamente disponibilizado.  

 **Checklist mínimo por questão**
- Código funcional, com comentários.  
- Tabelas/estatísticas corretas.  
- Gráficos legíveis (títulos, rótulos).  
- Resposta textual objetiva ao final de cada questão.

  **Rubrica de Avaliação**

- Correção técnica (40%) – código funcional e resultados corretos.
- Clareza e organização (20%) – código comentado e estruturado.
- Visualização (20%) – gráficos claros, bem rotulados e interpretados.
- Análise crítica (20%) – justificativas e interpretações consistentes.

# **1. Utilizando o microdados do ENEM verifique se as notas de matemática (NU_NOTA_MT), para cada estado, podem ser aproximada por uma distribuição Normal.**
   (Dica: use o teste de Shapiro-Wilk ou Kolmogorov-Smirnov em uma amostra)..

In [7]:
import pandas as pd
import scipy.stats as stats

path = '/Users/cleitonfraga/Documents/ciencia de dados/R/microdados_enem_nordeste.csv'
enem_df = pd.read_csv(path, sep=';', encoding='utf-8', encoding_errors='ignore')

# convertendo a coluna NU_NOTA_MT para numérica, tratando erros
enem_df['NU_NOTA_MT'] = pd.to_numeric(enem_df['NU_NOTA_MT'], errors='coerce')

dados_filtrados_estado = enem_df[["SG_UF_PROVA", "NU_NOTA_MT"]].copy()

# removendo linhas com valores nulos
dados_filtrados_estado.dropna(inplace=True)

# Função para aplicar o teste de Kolmogorov-Smirnov por estado e por tamanho de amostra
def teste_normalidade_por_estado(df, sample_size=5000):
    resultados = {}
    estados = df['SG_UF_PROVA'].unique()
    for estado in estados:
        notas_estado = df[df['SG_UF_PROVA'] == estado]['NU_NOTA_MT']
        n = len(notas_estado)
        # Aplicando o teste de Kolmogorov-Smirnov
        if n >= 3:
            # Amostragem para estados com muitos dados
            if n > sample_size:
                notas_estado = notas_estado.sample(sample_size, random_state=42)
            notas_normalizadas = (notas_estado - notas_estado.mean()) / notas_estado.std()
            ks_stat, p_value = stats.kstest(notas_normalizadas, 'norm')
            resultados[estado] = {'KS Statistic': ks_stat, 'p-value': p_value, 'n': n}
        else:
            resultados[estado] = {'KS Statistic': None, 'p-value': None, 'n': n}
    return pd.DataFrame(resultados).T

resultados_normalidade = teste_normalidade_por_estado(dados_filtrados_estado)
print(resultados_normalidade)




    KS Statistic        p-value        n
PA      0.151794  4.737376e-101  16356.0
MA      0.138307   6.788141e-84  11893.0
BA      0.118933   4.397356e-62  23001.0
SE      0.115169   1.134744e-56   4865.0
CE      0.107452   9.978149e-51  18010.0
RN      0.105558   5.763194e-49   7654.0
PB      0.119534   1.039210e-62   9172.0
AL      0.121468   9.519529e-65   5679.0
PI      0.122785   3.733832e-66   7256.0


#**Questão 2:**

Utilizando o microdados do ENEM, faça um comparativo entre a médias das notas
de matemática (NU_NOTA_MT) e verifique, por estado, se houve diferença
significativa entre os estados com nível de confiança de 90, 95 e 99%. Apresente
os resultados em forma de tabela.

In [9]:
import pandas as pd
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

path = '/Users/cleitonfraga/Documents/ciencia de dados/R/microdados_enem_nordeste.csv'

enem_df = pd.read_csv(path, sep=';', encoding='utf-8', encoding_errors='ignore')

# convertendo a coluna NU_NOTA_MT para numérica, tratando erros
enem_df['NU_NOTA_MT'] = pd.to_numeric(enem_df['NU_NOTA_MT'], errors='coerce')

# Filtrando dados relevantes
dados_filtrados_estado = enem_df[["SG_UF_PROVA", "NU_NOTA_MT"]].dropna()

# ANOVA
estados = dados_filtrados_estado['SG_UF_PROVA'].unique()
grupos = [dados_filtrados_estado[dados_filtrados_estado['SG_UF_PROVA'] == estado]['NU_NOTA_MT'] for estado in estados]
f_stat, p_anova = stats.f_oneway(*grupos)
print(f"ANOVA F-statistic: {f_stat:.4f}, p-value: {p_anova:.4g}")

# Tukey HSD
tukey = pairwise_tukeyhsd(endog=dados_filtrados_estado['NU_NOTA_MT'],
                          groups=dados_filtrados_estado['SG_UF_PROVA'],
                          alpha=0.10)  # 90% confianca
tukey_95 = pairwise_tukeyhsd(endog=dados_filtrados_estado['NU_NOTA_MT'],
                             groups=dados_filtrados_estado['SG_UF_PROVA'],
                             alpha=0.05)  # 95% confianca
tukey_99 = pairwise_tukeyhsd(endog=dados_filtrados_estado['NU_NOTA_MT'],
                             groups=dados_filtrados_estado['SG_UF_PROVA'],
                             alpha=0.01)  # 99% confianca

# Convertendo resultados do Tukey para DataFrame
def tukey_to_df(tukey_result, conf):
    df = pd.DataFrame(data=tukey_result._results_table.data[1:], columns=tukey_result._results_table.data[0])
    df['Significant at {}%'.format(int((1-conf)*100))] = df['reject']
    return df[['group1', 'group2', 'meandiff', 'p-adj', 'Significant at {}%'.format(int((1-conf)*100))]]

tabela_90 = tukey_to_df(tukey, 0.10)
tabela_95 = tukey_to_df(tukey_95, 0.05)
tabela_99 = tukey_to_df(tukey_99, 0.01)

print("\nComparações com 90% de confiança:")
print(tabela_90)
print("\nComparações com 95% de confiança:")
print(tabela_95)
print("\nComparações com 99% de confiança:")
print(tabela_99)


ANOVA F-statistic: 113.1344, p-value: 3.141e-189

Comparisons at 90% confidence:
   group1 group2  meandiff   p-adj  Significant at 90%
0      AL     BA   -6.2081  0.1990               False
1      AL     CE   13.9108  0.0000                True
2      AL     MA  -21.6357  0.0000                True
3      AL     PA  -29.1793  0.0000                True
4      AL     PB    4.7943  0.7203               False
5      AL     PI   -0.3730  1.0000               False
6      AL     RN   15.5384  0.0000                True
7      AL     SE    3.5234  0.9734               False
8      BA     CE   20.1189  0.0000                True
9      BA     MA  -15.4276  0.0000                True
10     BA     PA  -22.9712  0.0000                True
11     BA     PB   11.0024  0.0000                True
12     BA     PI    5.8350  0.1627               False
13     BA     RN   21.7465  0.0000                True
14     BA     SE    9.7315  0.0049                True
15     CE     MA  -35.5465  0.0000     

#**Questão 3. Faça uma comparação entre os estados da BA, SE, AL, PE, PB, RN, CE. E avalie se, em
média, o desempenho dos alunos em matemática teve diferença significativa. Se não teve,
identifique quais estados não o desempenho dos alunos em matemática não tiveram
diferença significativas (utilize um grau de confiança de 95 e 99%)**

In [10]:
import pandas as pd
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Carregar e limpar os dados
path = '/Users/cleitonfraga/Documents/ciencia de dados/R/microdados_enem_nordeste.csv'
enem_df = pd.read_csv(path, sep=';', encoding='utf-8', encoding_errors='ignore')
enem_df['NU_NOTA_MT'] = pd.to_numeric(enem_df['NU_NOTA_MT'], errors='coerce')

# Filtrar apenas os estados desejados
estados_selecionados = ['BA', 'SE', 'AL', 'PE', 'PB', 'RN', 'CE']
df_estados = enem_df[enem_df['SG_UF_PROVA'].isin(estados_selecionados)][['SG_UF_PROVA', 'NU_NOTA_MT']].dropna()

# ANOVA
grupos = [df_estados[df_estados['SG_UF_PROVA'] == estado]['NU_NOTA_MT'] for estado in estados_selecionados]
f_stat, p_anova = stats.f_oneway(*grupos)
print(f"ANOVA F-statistic: {f_stat:.4f}, p-value: {p_anova:.4g}")

# Tukey HSD para 95% e 99%
tukey_95 = pairwise_tukeyhsd(endog=df_estados['NU_NOTA_MT'],
                             groups=df_estados['SG_UF_PROVA'],
                             alpha=0.05)
tukey_99 = pairwise_tukeyhsd(endog=df_estados['NU_NOTA_MT'],
                             groups=df_estados['SG_UF_PROVA'],
                             alpha=0.01)

def tukey_to_df(tukey_result, conf):
    df = pd.DataFrame(data=tukey_result._results_table.data[1:], columns=tukey_result._results_table.data[0])
    df['Significativo a {}%'.format(int((1-conf)*100))] = df['reject']
    return df[['group1', 'group2', 'meandiff', 'p-adj', 'Significativo a {}%'.format(int((1-conf)*100))]]

tabela_95 = tukey_to_df(tukey_95, 0.05)
tabela_99 = tukey_to_df(tukey_99, 0.01)

print("\nComparações com 95% de confiança:")
print(tabela_95)
print("\nComparações com 99% de confiança:")
print(tabela_99)


/Users/cleitonfraga/PyCharmMiscProject/.venv/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4102: DegenerateDataWarning: at least one input has length 0
  if _f_oneway_is_too_small(samples):


ANOVA F-statistic: nan, p-value: nan

Comparações com 95% de confiança:
   group1 group2  meandiff   p-adj  Significativo a 95%
0      AL     BA   -6.2081  0.1180                False
1      AL     CE   13.9108  0.0000                 True
2      AL     PB    4.7943  0.5262                False
3      AL     RN   15.5384  0.0000                 True
4      AL     SE    3.5234  0.8875                False
5      BA     CE   20.1189  0.0000                 True
6      BA     PB   11.0024  0.0000                 True
7      BA     RN   21.7465  0.0000                 True
8      BA     SE    9.7315  0.0028                 True
9      CE     PB   -9.1165  0.0003                 True
10     CE     RN    1.6276  0.9799                False
11     CE     SE  -10.3873  0.0015                 True
12     PB     RN   10.7441  0.0004                 True
13     PB     SE   -1.2709  0.9981                False
14     RN     SE  -12.0150  0.0011                 True

Comparações com 99% de confianç